# Modeling Notes for algae bloom competition

By [Andrew Wheeler](mailto:apwheele@gmail.com), [Personal Blog to see my work](https://andrewpwheeler.com/)

These are my notes for the data competition, [Tick Tick Bloom: Harmful Algal Bloom Detection Challenge](https://www.drivendata.org/competitions/143/tick-tick-bloom/page/649/).

For a high level, I ensemble three different boosted models in the final solution:

  - xgboost with fields `region, cluster, date`
  - catboost with fields `region, cluster, date, latitude, longitude, maxe, dife`
  - lightboost with fields `region, cluster, imtype, date, latitude, longitude , elevation ,dife, imtype, prop_lake_2500, r_2500, g_2500, b_2500, prop_lake_1000, r_1000, g_1000, b_1000`

For a description of the variable fields:

  - region: metadata given region of US
  - cluster: arbitrary binning of lat/lon into different clusters (based on looking at distribution of lat/lon)
  - date: date field for event, the models under the hood turn these into features of days since 1/1/2015, day of the week, and month of the year (experimented with year as well, but not in final model)
  - latitude: given via metadata
  - latitude: given via metadata
  - maxe: maximum elevation within 1000 meters of lat/lon (from DEM)
  - dife: difference in min/max elevation of 1000 meters (from DEM)
  - elevation: elevation at exact lat/lon (from DEM)
  - imtype: image type (ended up only using Sentinel images, filtered out landsat entirely)
  - prop_lake_2500: Estimate of water area at 2500 meters from lat/lon
  - r_2500: estimate of red inside water area at 2500 meters
  - g_2500: estimate of green inside water area at 2500 meters
  - b_2500: estimate of blue inside water area at 2500 meters
  - prop_lake_1000: Estimate of water area at 1000 meters from lat/lon
  - r_1000: estimate of red inside water area at 1000 meters
  - g_1000: estimate of green inside water area at 1000 meters
  - b_1000: estimate of blue inside water area at 1000 meters 
  

In the competition, I started with each individual model, doing slight hyper-parameter tuning and seeing the optimal results on the leaderboad. Then I additionally tinkered with the final results (minor things, like tweaking the iterations and the depth of the trees).

I did not bother to download the weather data (just to download the satellite data was quite a chore). For future competitions I would suggest DataDriven provide a set of already produced functions, so it is easier for participants to understand what data is valid, and make it easier for people to participate. While I imagine weather data would improve the results, I did not have time to fully understand that data and incorporate it into my model.

Also note, given the error metric is mean-squared-error, I use a regression models, not multinomial. I then predict a continuous outcome, and then round/clip to the nearest integer value. For the ensemble models, I take a simple average of the not rounded values, and then round the final result.

For the satellite imagery, I used [k-means image segmentation](https://docs.opencv.org/3.4/d1/d5c/tutorial_py_kmeans_opencv.html) to try to identify water areas (I doubt this works very well, the other model features did most of the work), and then calculate the R/G/B stats, as well as the size of the water area. The final models I chose within 1000 meters and 2500 meters doing this, but also tested within 500 meters as well.

**NOTE**

I initially made a mistake, and used sentinel data + landsat-7. The solution on 2/6 (private score 0.7580) incorrectly used the landsat-7 data. So my 2nd best solution on 2/16 (private score 0.7616) is what I illustrate here.

It is a stupid mistake and I apologize. For those wishing to replicate the same results but with LandSat-8 data, you just need to change 1 character in `src/sat_fe.py` on line 89.

**END NOTE**

I think it would be better to just use [NOAA's list of lakes to filter the imagery](https://community.drivendata.org/t/outside-data-sources/8325/2), but that was against the competition rules. (Smaller lakes have higher concentrations.) Also this model will be bad if you submit somewhere with no lake to begin with (it will spit out numbers, but obviously they should be no risk of bloom). Using the lakes data would limit where it makes sense to even make predictions.

In [1]:
# Personal functions, these should be run from ./algaebloom, see the README.md
from src import feat, mod
from src.single_fe import get_features, pred_out
import pandas as pd

# Show function for predicting single out of sample

# Grabbing the data used for training
train = feat.get_data(split_pred=True)
test = feat.get_data(data_type='test') # Note that this is the submission format data
train.head()

,uid,region,severity,density,latitude,longitude,date,elevation,mine,maxe,...,r_1000,g_1000,b_1000,prop_lake_2500,r_2500,g_2500,b_2500,cluster,logDensity,split_pred
0,aabm,3,1,585.0,39.080319,-86.430867,2018-05-14,164.000000,162.775070,245.746124,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,5,6.371612,0.026089
1,aacd,2,1,290.0,35.875083,-78.878434,2020-11-19,112.367500,82.372543,124.047012,...,167.504627,155.254219,135.638269,0.160221,158.483822,143.132806,120.706959,2,5.669881,0.003110
2,aaee,2,1,1614.0,35.487000,-79.062133,2016-08-24,133.678329,94.498573,160.707916,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2,7.386471,0.002955
3,aaff,3,3,111825.0,38.049471,-99.827001,2019-07-23,693.751953,685.274170,726.738892,...,229.735649,188.723331,146.595467,0.102113,205.936458,168.080443,128.281072,6,11.624690,0.027916
4,aafl,3,4,2017313.0,39.474744,-86.898353,2021-08-23,199.500000,199.068390,271.521088,...,238.247501,242.393052,239.020702,0.252126,243.033961,245.751794,243.296854,5,14.517277,0.140605


## EDA

Some EDA notes I have. First, the spatial variation is quite substantial. In real production, I would want to more formally model this (maybe via a kriging or other type of spatial auto-regressive/spatial error model). But given the competition has spatial-hold out sets, this is not viable.

A simple approach, which improved the models quite a bit, was to create an ad-hoc cluster variable, to identify cluster areas for the test set data (but identify train set close them in space). I represented this variable via the `cluster` ordinal variable.

In [2]:
# Show region/cluster variability
print('Here is how I encoded regions')
print(feat.reg_ord)
reg_stats = train.groupby(['region'],as_index=False)[['severity']].describe()
print(reg_stats)

print("\nHere are my cluster differences")
clus_stats = train.groupby(['cluster'],as_index=False)[['severity']].describe()
clus_stats

Here is how I encoded regions
{'west': 4, 'midwest': 3, 'south': 2, 'northeast': 1}
  severity                                             
     count      mean       std  min  25%  50%  75%  max
0   1143.0  1.805774  0.938980  1.0  1.0  2.0  2.0  5.0
1   9948.0  1.567652  0.783326  1.0  1.0  1.0  2.0  5.0
2   2200.0  2.194091  1.043424  1.0  1.0  2.0  3.0  5.0
3   3769.0  3.747413  0.715230  1.0  4.0  4.0  4.0  5.0

Here are my cluster differences


severity                                             
     count      mean       std  min  25%  50%  75%  max
0    492.0  1.666667  0.983917  1.0  1.0  1.0  2.0  5.0
1   9565.0  1.551281  0.770109  1.0  1.0  1.0  2.0  5.0
2    812.0  1.837438  0.992290  1.0  1.0  2.0  3.0  5.0
3    207.0  1.685990  0.732695  1.0  1.0  2.0  2.0  4.0
4   1507.0  1.972130  1.017388  1.0  1.0  2.0  3.0  5.0
5   1044.0  2.576628  0.959821  1.0  2.0  3.0  3.0  5.0
6   3433.0  3.917565  0.321922  1.0  4.0  4.0  4.0  5.0

One additional thing I want to show, is that there are significant differences across *days of the week*. Clearly this does not matter for mitosis, so this suggests some type of human based biased in the data. It would be useful to set a randomized set of days for data collection going forward in the future, as this is clearly an artificial thing that may work well for the competition, but will not likely work well in real data.

In [3]:
# Show differences across day of week (mitosis)

# 0 is Monday, 7 is Sunday

bt = pd.to_datetime('1/1/2015')
diff_days, week_day, month, year = mod.dummy_stats(train['date'],begin_date=bt)
train['week_day'] = week_day
week_stats = train.groupby(['week_day'],as_index=False)[['severity']].describe()
week_stats  # Tuesdays/Fridays are worse!

severity                                             
     count      mean       std  min  25%  50%  75%  max
0   3332.0  2.111044  1.060595  1.0  1.0  2.0  3.0  5.0
1   5407.0  2.588496  1.320560  1.0  1.0  3.0  4.0  5.0
2   4837.0  1.824271  1.069023  1.0  1.0  1.0  3.0  5.0
3   2853.0  1.899755  1.046823  1.0  1.0  2.0  3.0  5.0
4    380.0  2.336842  1.328483  1.0  1.0  2.0  4.0  5.0
5    131.0  1.786260  1.109453  1.0  1.0  1.0  2.5  4.0
6    120.0  1.783333  1.022300  1.0  1.0  1.0  3.0  4.0

## Train/Test Validation Approach

So because the test data is totally different lat/lon coordinates (spatial hold outs), it was important to identify the differences in the two samples. Using this, I was able to use a hyper-tuning strategy that much better approximated the real test data using in-sample data.

The variable `split_pred`, in the above dataset was generated via predicing the probability a sample is in the test set based on the variables `latitude,longitude,maxe,dife,region`. Here I show generating a second example with the same approach as in `get_split.py`.

In [4]:
# Show pred training
all_dat = feat.get_both() # this is the combined train/test data

# Predicting probability of in test set
cm = mod.CatMod(ord_vars=['region'],
                ide_vars=['latitude','longitude','maxe','dife'],
                y='test')
cm.fit(all_dat)

# Predict probability in the train set
split_pred2 = cm.predict(train)
train['split_pred2'] = split_pred2
train[['uid','split_pred','split_pred2']].head()

,uid,split_pred,split_pred2
0,aabm,0.026089,0.026089
1,aacd,0.003110,0.003110
2,aaee,0.002955,0.002955
3,aaff,0.027916,0.027916
4,aafl,0.140605,0.140605


Using a typical train/test approach, I get much too optimistic results. Here I do not use weights to determine the train/test splits. (Under the hood this does random test sizes of 2000 and does 10 random train/test splits.)

In [5]:
# Show off differences in train/test with no weights

lig = mod.RegMod(ord_vars=['region','cluster','imtype'],
                dat_vars=['date'],
                ide_vars=['latitude','longitude','elevation','dife'],
                y='severity',
                weight='split_pred2',
                mod = mod.LGBMRegressor(n_estimators=1000,max_depth=12)
                )

# Waaaay too optimistic, overall average MSE 0.65
met_noweights = lig.met_eval(train,pr=True,ret=True,weight=False,cat=False,full_train=False,split_tt='not_weighted')

           count      mean       std       min       25%       50%       75%  \
AvgError    10.0  0.651469  0.029104  0.605766  0.635764  0.651449  0.661734   
midwest     10.0  0.724845  0.034729  0.659656  0.701418  0.731358  0.754869   
northeast   10.0  0.741491  0.091785  0.613941  0.675947  0.767488  0.784505   
south       10.0  0.755994  0.011216  0.740193  0.746156  0.756982  0.762152   
west        10.0  0.383546  0.031974  0.340047  0.362920  0.382151  0.403413   

                max  
AvgError   0.715457  
midwest    0.761183  
northeast  0.909718  
south      0.775365  
west       0.437079  


In [6]:
# Using the same model, but with weights for the train/test split evaluations
# is much more realistic compared to actual leaderboard

# This model has an average MSE of 0.79, much closer to real life (this one is overfit)
met_weights = lig.met_eval(train,pr=True,ret=True,weight=True,cat=False,full_train=False,split_tt='weighted')

           count      mean       std       min       25%       50%       75%  \
AvgError    10.0  0.798991  0.019836  0.765394  0.788205  0.795500  0.811945   
midwest     10.0  0.832061  0.027888  0.791447  0.816570  0.829026  0.846358   
northeast   10.0  0.797087  0.037476  0.742641  0.772045  0.797776  0.809876   
south       10.0  0.798426  0.026542  0.744133  0.782781  0.804539  0.819512   
west        10.0  0.768390  0.041207  0.707966  0.752090  0.768277  0.787684   

                max  
AvgError   0.829185  
midwest    0.890946  
northeast  0.856819  
south      0.829156  
west       0.840987  


In tinkering with the leaderboard, here is the final set of ensembled models. I tested with individual models up until January 4th. Individual models have at best RMSE's around 0.8 (best one was 0.78 with a very simple xgboost model without satellite data). But ensembling the models together resulted in a drop to 0.76. Then I just tinkered with a few things (iterations, depth, changing single variables) to drop down below 0.75 over time.

In [7]:
# Showing off fitting the final model

sat_1000 = ['prop_lake_1000', 'r_1000', 'g_1000', 'b_1000']
sat_2500 = ['prop_lake_2500', 'r_2500', 'g_2500', 'b_2500']
sat_1025 = sat_1000 + sat_2500

cat = mod.RegMod(ord_vars=['region','cluster'],
                dat_vars=['date'],
                ide_vars=['latitude','longitude','maxe','dife'],
                y='severity',
                mod = mod.CatBoostRegressor(iterations=380,depth=6,
                   allow_writing_files=False,verbose=False)
                )
cat.fit(train,weight=False,cat=False)


lig = mod.RegMod(ord_vars=['region','cluster','imtype'],
                dat_vars=['date'],
                ide_vars=['latitude','longitude','elevation','dife'] + sat_1025,
                y='severity',
                mod = mod.LGBMRegressor(n_estimators=470,max_depth=8)
                )
lig.fit(train,weight=False,cat=True)


xgb = mod.RegMod(ord_vars=['region','cluster'],
                 dat_vars=['date'],
                 y='severity',
                 mod = mod.XGBRegressor(n_estimators=70, max_depth=2))
xgb.fit(train,weight=False,cat=False)


rm = mod.EnsMod(mods={'xgb': xgb, 'cat': cat, 'lig': lig})


# To generate predictions for the test set
test['pred'] = rm.predict_int(test)
form_dat = feat.sub_format(test)
print('Distribution of predictions for test set')
print(form_dat['severity'].value_counts())

# Showing it is equivalent to other saved model
current = form_dat.copy()
print('\nThis shows differences in current model vs best results on 2/16')
print(mod.check_day(current,day="sub_2023_02_16.csv")) # should all be 0's if the model is the same


# To save the final sample
form_dat.to_csv(f'sub_notebook.csv',index=False)

# And to save the model if you want
# default saves to the ./model folder
mod.save_model(rm,f'mod_notebook')

Distribution of predictions for test set
2    2512
4    2091
1    1241
3     666
Name: severity, dtype: int64

This shows differences in current model vs best results on 2/16
0    6510
Name: dif_2023_02_16, dtype: int64
None


In terms of model interpretability, this is quite low. It is an ensembled model of many different boosted trees, so can be highly non-linear and factor in many different variables. Here is a quick view of each models normalized feature importance (need to see the documents for each model, as they are all slightly different).

In [8]:
print('Catboost Feature Importance')
print(cat.feat_import())

print('\nLightboost Feature Importance')
print(lig.feat_import())

print('\nXGBoost Feature Importance')
print(xgb.feat_import())

Catboost Feature Importance
            Var        FI
0       cluster  0.174895
1     longitude  0.166795
2      latitude  0.157506
3          maxe  0.125172
4     days_date  0.117256
5          dife  0.104067
6    month_date  0.094194
7        region  0.035100
8  weekday_date  0.025013

Lightboost Feature Importance
               Var        FI
0        elevation  0.157693
1        days_date  0.150518
2             dife  0.126096
3         latitude  0.113921
4        longitude  0.106747
5       month_date  0.053917
6   prop_lake_1000  0.051960
7   prop_lake_2500  0.043699
8           r_1000  0.039858
9           b_1000  0.028698
10          g_1000  0.028408
11          r_2500  0.026886
12          b_2500  0.023915
13          g_2500  0.021741
14    weekday_date  0.016233
15         cluster  0.007392
16          imtype  0.001667
17          region  0.000652

XGBoost Feature Importance
            Var        FI
0       cluster  0.889193
1    month_date  0.052532
2        region  0.04181

Note because only one of these models has satellite imagery data, the solution is mostly just fitting a curve based on dates over time and spatial variation. In practice this model is not likely to have much more discriminative ability than to just say "water areas in this area at this time tend to have these concentrations".

It is very difficult to create a competition that effectively allows one to build a model that can be actually used in production. I think it would need to be submitting a function, which can leverage prior nearby samples in space-time to generate predictions, and you apply it to totally new samples in the future.

## Showing Prediction for a single new observation

This is to illustrate how to make a prediction for a single piece of data given lat/lon/date/region and piping in a particular saved model. So here I show for the model built in this exact notebook in earlier cells.

In [9]:
# Show prediction

aabn = test[['latitude','longitude','date','region','pred']].head(1).values.tolist()[0] #aabn

# The function expects region to be the original string
rev_ord = {v:k for k,v in feat.reg_ord.items()}
aabn[3] = rev_ord[aabn[3]]
print('Vector and prediction from original model')
print(aabn)

print('\nPrediction grabbing data right now and using local model')
print(pred_out(aabn[0],aabn[1],aabn[2],aabn[3],rm))

Vector and prediction from original model
[36.5597, -121.51, '2016-08-31', 'west', 4]

Prediction grabbing data right now and using local model
4


In [10]:
# Show saving values, and show it is the same as single row
vars_tocheck = ['cluster','imtype','elevation','dife','maxe'] + sat_1025

res_vals = get_features(aabn[0],aabn[1],aabn[2],aabn[3])
print('Here is the dynamically generated feature values')
for v in vars_tocheck:
    print(f'{v}: {res_vals[v]}')

print('\nHere is the original feature values in the cached dataset')
vars_tocheck = ['cluster','imtype','elevation','dife','maxe'] + sat_1025
test[vars_tocheck].head(1).T

# I sometimes get minor differences, I presume this could be due to the init
# for the k-means functions

Here is the dynamically generated feature values
cluster: 7
imtype: 1
elevation: 30.869382858276367
dife: 21.346538543701172
maxe: 49.59114074707031
prop_lake_1000: 0.24529027297193387
r_1000: 153.4611089341693
g_1000: 116.86539968652038
b_1000: 79.04202586206897
prop_lake_2500: 0.27616292560679423
r_2500: 151.63212805287924
g_2500: 119.83840746134887
b_2500: 77.18650571364553

Here is the original feature values in the cached dataset


,0
cluster,7.000000
imtype,1.000000
elevation,30.869383
dife,21.346539
maxe,49.591141
prop_lake_1000,0.245098
r_1000,155.874804
g_1000,120.173824
b_1000,83.218529
prop_lake_2500,0.275378


## Duan Smearing predicting continous outcomes

I think you should be predicting the continuous outcomes, and then can reduce them into bins if you want (or make a smarter threshold determination). So I think you should predict the density directly, not predict the intermediate reduced set of 1 to 5 rankings. 

I wrote code to predict the logDensity, and then put them back into the original density scale using [Duan's smearing](https://andrewpwheeler.com/2021/02/19/transforming-predicted-variables-in-regression/). 

This ended up being worse though in the competition than just predicing the integer severities 1-5 though in my experiments. So I include here just to show it off. (Hit me up NASA/NOAA if you need some stat consulting!) Not shown here, you may also want to [generate prediction intervals](https://andrewpwheeler.com/2022/02/04/prediction-intervals-for-random-forests/), and only flag if the low end of the interval is above some threshold.

In [11]:
lig_logDens = mod.RegMod(ord_vars=['region','cluster','imtype'],
                         dat_vars=['date'],
                         ide_vars=['latitude','longitude','elevation','dife'] + sat_1025,
                         y='density',
                         transform=mod.safelog,
                         inv_trans=mod.np.exp,
                         mod = mod.LGBMRegressor(n_estimators=500,max_depth=8)
                         )

lig_logDens.fit(train,weight=False,cat=True)

test['predDens'] = lig_logDens.predict(test,duan=True)
test['logDens'] = lig_logDens.predict(test,duan=False)

# pred is the integer prediction, 
# predDens is the density prediction
# logDens is the prediction on log scale
test[['uid','pred','predDens','logDens']].head(10)

,uid,pred,predDens,logDens
0,aabn,4,1.438460e+07,15.472339
1,aair,4,3.597007e+06,14.086283
2,aajw,2,3.998125e+04,9.586837
3,aalr,3,1.303134e+06,13.070954
4,aalw,4,1.954046e+06,13.476083
5,aamp,3,1.043011e+06,12.848293
6,aapj,4,3.074452e+06,13.929308
7,aaqf,1,1.635582e+04,8.693010
8,aauy,1,1.190777e+04,8.375617
9,aava,1,6.737680e+02,5.503557
